In [38]:
# pip install wikipedia-api

In [39]:
import pandas as pd
import requests
from time import sleep
from tqdm import tqdm

In [40]:
df = pd.read_csv("female_artists_with_work_counts.csv")
df

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN,artwork_count,artwork_range
0,10,Irene Aronson,"American, born Germany 1918",American,female,1918,0,Q19748568,500042413.0,1.0,1
1,21,Ruth Asawa,"American, 1926–2013",American,female,1926,2013,Q7382874,500077806.0,56.0,51-100
2,22,Isidora Aschheim,Israeli,Israeli,female,0,0,NaN,NaN,1.0,1
3,28,Geneviève Asse,"French, born 1923",French,female,1923,0,Q1978192,500025324.0,50.0,11-50
4,31,Dana Atchley,"American, 1941–2000",American,female,1941,2000,NaN,NaN,7.0,6-10
...,...,...,...,...,...,...,...,...,...,...,...
2430,137853,Silvia Rosi,"Italian and Togolese, born 1992",Italian,female,1992,0,NaN,NaN,6.0,6-10
2431,138111,Ulla Wiggen,"Swedish, born 1942",Swedish,female,1942,0,NaN,NaN,2.0,2-5
2432,138280,Teresita Brigitte Cochran,"American, born 1975",American,female,1975,0,NaN,NaN,1.0,1
2433,138366,Johanna Unzueta,"Chilean, born 1974 Santiago, Chile. Lives and ...",Chilean,female,1974,0,NaN,NaN,1.0,1


In [41]:
df_for_wiki = df[["DisplayName", "ArtistBio","Wiki QID", "artwork_count"]]

In [42]:
df_for_wiki['Wiki QID'].isna().sum()


1918

In [43]:
df_for_wiki

,DisplayName,ArtistBio,Wiki QID,artwork_count
0,Irene Aronson,"American, born Germany 1918",Q19748568,1.0
1,Ruth Asawa,"American, 1926–2013",Q7382874,56.0
2,Isidora Aschheim,Israeli,NaN,1.0
3,Geneviève Asse,"French, born 1923",Q1978192,50.0
4,Dana Atchley,"American, 1941–2000",NaN,7.0
...,...,...,...,...
2430,Silvia Rosi,"Italian and Togolese, born 1992",NaN,6.0
2431,Ulla Wiggen,"Swedish, born 1942",NaN,2.0
2432,Teresita Brigitte Cochran,"American, born 1975",NaN,1.0
2433,Johanna Unzueta,"Chilean, born 1974 Santiago, Chile. Lives and ...",NaN,1.0


In [44]:
2435-1918 #1918 dont have wiki . remain 517 have wiki links 

517

In [45]:
df_for_wiki['Wiki QID'] = df_for_wiki['Wiki QID'].astype(str)

/var/folders/2_/mdjrqzks7tj2_q8gf6cl0s6w0000gn/T/ipykernel_69459/4189516996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_wiki['Wiki QID'] = df_for_wiki['Wiki QID'].astype(str)


In [46]:
def fetch_wikipedia_summary(qid):
    """
    Fetch Wikipedia summary and URL for a given QID.
    """
    api_url = "https://www.wikidata.org/w/api.php"
    headers = {
        'User-Agent': 'WikiDataSummaryBot/1.0 (your@email.com)'  # Replace with your email
    }
    
    try:
        # Fetch Wikidata entity details
        params = {
            "action": "wbgetentities",
            "ids": qid,
            "format": "json"
        }
        
        response = requests.get(api_url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        if qid in data.get('entities', {}):
            entity = data['entities'][qid]
            # Get English Wikipedia URL
            sitelinks = entity.get('sitelinks', {})
            enwiki = sitelinks.get('enwiki', {})
            wiki_url = enwiki.get('url', "")
            
            # Fetch summary from Wikipedia
            if wiki_url:
                title = enwiki.get('title', "").replace(' ', '_')
                wikipedia_api_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
                wiki_response = requests.get(wikipedia_api_url, headers=headers)
                wiki_response.raise_for_status()
                wiki_data = wiki_response.json()
                summary = wiki_data.get("extract", "")
                return summary, wiki_url
            
    except Exception as e:
        print(f"Error fetching data for QID {qid}: {e}")
    
    return None, None

In [48]:
summaries = []
links = []

for _, row in tqdm(df_for_wiki.iterrows(), total=len(df_for_wiki), desc="Fetching Wikipedia data"):
    qid = row['Wiki QID']
    if pd.notnull(qid) and qid.startswith("Q"):
        summary, link = fetch_wikipedia_summary(qid)
        summaries.append(summary)
        links.append(link)
        sleep(1)  # Be kind to the API
    else:
        summaries.append(None)
        links.append(None)

Fetching Wikipedia data: 100%|██████████| 2435/2435 [10:47<00:00,  3.76it/s]


In [49]:
df_for_wiki.loc[:,'wiki_summary'] = summaries
df_for_wiki.[:,'wiki_link'] = links

/var/folders/2_/mdjrqzks7tj2_q8gf6cl0s6w0000gn/T/ipykernel_69459/1725965792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_wiki['wiki_summary'] = summaries
/var/folders/2_/mdjrqzks7tj2_q8gf6cl0s6w0000gn/T/ipykernel_69459/1725965792.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_wiki['wiki_link'] = links


In [50]:
# for index, row in df_for_wiki.iterrows():
#     qid = row['Wiki QID']
#     if pd.notnull(qid):  # Skip rows with null QIDs
#         # summary, link = fetch_wiki_data(qid)
#         df_for_wiki.at[index, 'wiki_summary'] = summary
#         df_for_wiki.at[index, 'wiki_link'] = link


In [51]:
df_for_wiki

,DisplayName,ArtistBio,Wiki QID,artwork_count,wiki_summary,wiki_link
0,Irene Aronson,"American, born Germany 1918",Q19748568,1.0,None,None
1,Ruth Asawa,"American, 1926–2013",Q7382874,56.0,None,None
2,Isidora Aschheim,Israeli,nan,1.0,None,None
3,Geneviève Asse,"French, born 1923",Q1978192,50.0,None,None
4,Dana Atchley,"American, 1941–2000",nan,7.0,None,None
...,...,...,...,...,...,...
2430,Silvia Rosi,"Italian and Togolese, born 1992",nan,6.0,None,None
2431,Ulla Wiggen,"Swedish, born 1942",nan,2.0,None,None
2432,Teresita Brigitte Cochran,"American, born 1975",nan,1.0,None,None
2433,Johanna Unzueta,"Chilean, born 1974 Santiago, Chile. Lives and ...",nan,1.0,None,None


In [52]:
df_for_wiki['wiki_summary'].isna().sum()


2435